In [512]:
#TO RE-RUN
%reset -f

In [513]:
from sklearn import preprocessing
from time import time
import numpy as np
import csv
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.cross_validation import StratifiedShuffleSplit, cross_val_score

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

from sklearn.grid_search import GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE,ADASYN, RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline

from operator import truediv
from sklearn import metrics
import pandas as pd
import time
import os

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('classic')

%matplotlib inline

import sys
sys.path.insert(1, "../../src/")
from TypeFeatImputer import TypeFeatImputer
from UnivCombineFilter import UnivCombineFilter

In [514]:
typeEncounter = "last" #["last","first"]

In [515]:
typeDiagnosis = "none" 

In [516]:
#df_all=pd.read_csv(os.path.join('resources','diabetic_data_processed_withweight.csv'),';')
df_all=pd.read_pickle(os.path.join('resources','clean_data_' + typeEncounter + '.pkl'))
print df_all.shape
print df_all.columns.tolist()
print df_all.readmitted.value_counts()
print df_all.readmitted.value_counts()/float(df_all.shape[0])

(67182, 194)
['gender', 'age', 'weight', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'Change', 'diabetesMed', 'Diabetis_3', 'Infectious and parasitic diseases_3', 'Neoplasms_3', 'Endocrine_3', 'Blood_3', 'Mental_3', 'Nervous_3', 'Organs_3', 'Circulatory_3', 'Respiratory_3', 'Digestive_3', 'Genitourinary_3', 'Pregnancy_3', 'Skin_3', 'Muscoskeletal_3', 'Congenital_3', 'Perinatal_3', 'Ill-defined_3', 'Poisoning_3', 'Diabetis_1', 'Infectious and parasitic diseases_1', 

### Features preparation

In [517]:
df_all["HbA1c"] = -1
df_all["HbA1c"][df_all.A1Cresult == 2] = 3
df_all["HbA1c"][df_all.A1Cresult == 1] = 2
df_all["HbA1c"][df_all.A1Cresult == 0] = 1
df_all["HbA1c"][df_all.A1Cresult.isnull()] = 0

pd.value_counts(df_all.HbA1c)

/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  afte

0    52657
3     6833
1     4397
2     3295
Name: HbA1c, dtype: int64

In [518]:
#Not useful
df_all["ComplexHbA1c"] = 0
df_all["ComplexHbA1c"][np.logical_and(df_all.A1Cresult == 2, df_all.Change == 1)] = 2
df_all["ComplexHbA1c"][np.logical_and(df_all.A1Cresult == 2, df_all.Change == 0)] = 1

pd.value_counts(df_all.ComplexHbA1c)

/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


0    60349
2     4395
1     2438
Name: ComplexHbA1c, dtype: int64

In [519]:
#Not useful
df_all["age_cat"] = -1
df_all["age_cat"][df_all.age.isin([0,1,2])] = 0
df_all["age_cat"][df_all.age.isin([3,4,5,6])] = 1
df_all["age_cat"][df_all.age.isin([7,8,9,10])] = 2

print pd.value_counts(df_all["age_cat"])

1    35869
2    29571
0     1742
Name: age_cat, dtype: int64


/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [520]:
#Not useful
df_all["adm_src_ref"] = np.sum(df_all[["adm_src_1","adm_src_2","adm_src_3"]].values,axis=1)>0
df_all["adm_src_em"] = df_all["adm_src_7"].values

print pd.value_counts(df_all["adm_src_ref"])
print pd.value_counts(df_all["adm_src_em"])

False    45345
True     21837
Name: adm_src_ref, dtype: int64
1    36050
0    31132
Name: adm_src_em, dtype: int64


In [521]:
df_all["medSpec_cardio"] = np.sum(
    df_all[['medSpec_Cardiology', 'medSpec_Cardiology-Pediatric']], axis=1)>0

pd.value_counts(df_all["medSpec_cardio"])

False    62857
True      4325
Name: medSpec_cardio, dtype: int64

In [522]:
df_all["medSpec_surgery"] = np.sum(
    df_all[['medSpec_Surgeon', 'medSpec_Surgery-Cardiovascular', 'medSpec_Surgery-Cardiovascular/Thoracic', 
            'medSpec_Surgery-Colon&Rectal', 'medSpec_Surgery-General', 'medSpec_Surgery-Maxillofacial', 
            'medSpec_Surgery-Neuro', 'medSpec_Surgery-Pediatric', 'medSpec_Surgery-Plastic', 
            'medSpec_Surgery-Thoracic', 'medSpec_Surgery-Vascular', 
            'medSpec_SurgicalSpecialty', 'medSpec_Surgery-PlasticwithinHeadandNeck']], axis=1)>0

pd.value_counts(df_all["medSpec_surgery"])

False    63383
True      3799
Name: medSpec_surgery, dtype: int64

In [523]:
df_all["number_treatment"] = -1
df_all["number_treatment"] = np.sum(df_all[[ u'metformin', u'repaglinide', u'nateglinide',
       u'chlorpropamide', u'glimepiride', u'acetohexamide', u'glipizide',
       u'glyburide', u'tolbutamide', u'pioglitazone', u'rosiglitazone',
       u'acarbose', u'miglitol', u'troglitazone', u'tolazamide', u'examide',
       u'citoglipton', u'insulin', u'glyburide-metformin',
       u'glipizide-metformin', u'glimepiride-pioglitazone',
       u'metformin-rosiglitazone', u'metformin-pioglitazone']], axis=1)

print np.unique(df_all["number_treatment"])
print pd.value_counts(df_all["number_treatment"])
print pd.value_counts(df_all["number_treatment"])/df_all.shape[0]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
2     20971
0     15699
4     10374
3      6594
1      4655
6      3494
5      3122
7      1268
8       678
9       230
10       68
11       21
12        7
13        1
Name: number_treatment, dtype: int64
2    0.31
0    0.23
4    0.15
3    0.10
1    0.07
6    0.05
5    0.05
7    0.02
8    0.01
9    0.00
10   0.00
11   0.00
12   0.00
13   0.00
Name: number_treatment, dtype: float64


In [524]:
df_all["number_treatment_0"] = -1
df_all["number_treatment_0"] = np.sum(df_all[[ u'metformin', u'repaglinide', u'nateglinide',
       u'chlorpropamide', u'glimepiride', u'acetohexamide', u'glipizide',
       u'glyburide', u'tolbutamide', u'pioglitazone', u'rosiglitazone',
       u'acarbose', u'miglitol', u'troglitazone', u'tolazamide', u'examide',
       u'citoglipton', u'insulin', u'glyburide-metformin',
       u'glipizide-metformin', u'glimepiride-pioglitazone',
       u'metformin-rosiglitazone', u'metformin-pioglitazone']]==0, axis=1)
print np.unique(df_all["number_treatment_0"])
print pd.value_counts(df_all["number_treatment_0"])
print pd.value_counts(df_all["number_treatment_0"])/df_all.shape[0]

df_all["number_treatment_1"] = -1
df_all["number_treatment_1"] = np.sum(df_all[[ u'metformin', u'repaglinide', u'nateglinide',
       u'chlorpropamide', u'glimepiride', u'acetohexamide', u'glipizide',
       u'glyburide', u'tolbutamide', u'pioglitazone', u'rosiglitazone',
       u'acarbose', u'miglitol', u'troglitazone', u'tolazamide', u'examide',
       u'citoglipton', u'insulin', u'glyburide-metformin',
       u'glipizide-metformin', u'glimepiride-pioglitazone',
       u'metformin-rosiglitazone', u'metformin-pioglitazone']]==1, axis=1)
print np.unique(df_all["number_treatment_1"])
print pd.value_counts(df_all["number_treatment_1"])
print pd.value_counts(df_all["number_treatment_1"])/df_all.shape[0]

df_all["number_treatment_2"] = -1
df_all["number_treatment_2"] = np.sum(df_all[[ u'metformin', u'repaglinide', u'nateglinide',
       u'chlorpropamide', u'glimepiride', u'acetohexamide', u'glipizide',
       u'glyburide', u'tolbutamide', u'pioglitazone', u'rosiglitazone',
       u'acarbose', u'miglitol', u'troglitazone', u'tolazamide', u'examide',
       u'citoglipton', u'insulin', u'glyburide-metformin',
       u'glipizide-metformin', u'glimepiride-pioglitazone',
       u'metformin-rosiglitazone', u'metformin-pioglitazone']]==2, axis=1)

print np.unique(df_all["number_treatment_2"])
print pd.value_counts(df_all["number_treatment_2"])
print pd.value_counts(df_all["number_treatment_2"])/df_all.shape[0]

df_all["number_treatment_3"] = -1
df_all["number_treatment_3"] = np.sum(df_all[[ u'metformin', u'repaglinide', u'nateglinide',
       u'chlorpropamide', u'glimepiride', u'acetohexamide', u'glipizide',
       u'glyburide', u'tolbutamide', u'pioglitazone', u'rosiglitazone',
       u'acarbose', u'miglitol', u'troglitazone', u'tolazamide', u'examide',
       u'citoglipton', u'insulin', u'glyburide-metformin',
       u'glipizide-metformin', u'glimepiride-pioglitazone',
       u'metformin-rosiglitazone', u'metformin-pioglitazone']]==3, axis=1)

print np.unique(df_all["number_treatment_3"])
print pd.value_counts(df_all["number_treatment_3"])
print pd.value_counts(df_all["number_treatment_3"])/df_all.shape[0]

[17 18 19 20 21 22 23]
22    29987
23    15699
21    14922
20     5536
19      987
18       49
17        2
Name: number_treatment_0, dtype: int64
22   0.45
23   0.23
21   0.22
20   0.08
19   0.01
18   0.00
17   0.00
Name: number_treatment_0, dtype: float64
[0 1 2 3]
0    58722
1     8278
2      179
3        3
Name: number_treatment_1, dtype: int64
0   0.87
1   0.12
2   0.00
3   0.00
Name: number_treatment_1, dtype: float64
[0 1 2 3 4 5 6]
1    25647
0    25251
2    12215
3     3557
4      494
5       17
6        1
Name: number_treatment_2, dtype: int64
1   0.38
0   0.38
2   0.18
3   0.05
4   0.01
5   0.00
6   0.00
Name: number_treatment_2, dtype: float64
[0 1 2 3]
0    58425
1     8348
2      391
3       18
Name: number_treatment_3, dtype: int64
0   0.87
1   0.12
2   0.01
3   0.00
Name: number_treatment_3, dtype: float64


In [525]:
print df_all.shape

df_all["diss_home"] = -1
df_all["diss_home"] = np.sum(df_all[['diss_1','diss_13']], axis=1)

print pd.value_counts(df_all["diss_home"])

(67182, 206)
1    44560
0    22622
Name: diss_home, dtype: int64


In [526]:
colsFinal = ['gender', 'age',"race_AfricanAmerican","race_Caucasian","race_Other", 
             'HbA1c',
             "Change",
             'time_in_hospital',
             'diabetesMed',
             "diss_home",             
             "medSpec_cardio","medSpec_Family/GeneralPractice","medSpec_InternalMedicine","medSpec_surgery",             
             'adm_src_1', 'adm_src_2', 'adm_src_3', 'adm_src_4', 'adm_src_5', 'adm_src_6', 'adm_src_7', 'adm_src_8', 
             'adm_src_10', 'adm_src_11', 'adm_src_13', 'adm_src_14', 'adm_src_22', 'adm_src_25',
              u'adm_1', u'adm_2', u'adm_3', u'adm_4', u'adm_7', 
             "number_treatment",
             'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 
             'number_emergency', 'number_inpatient', 'number_diagnoses']

colsFinalExtended = colsFinal[:]
colsFinalExtended.extend([
    "insulin","metformin","pioglitazone","glimepiride","glipizide","repaglinide","nateglinide"])
colsFinalExtendedExtra = colsFinalExtended[:]
colsFinalExtendedExtra.extend([
    "ComplexHbA1c",
    "number_treatment_0","number_treatment_1","number_treatment_2","number_treatment_3"])

colsFinal.extend(["readmitted"])
colsFinalExtended.extend(["readmitted"])
colsFinalExtendedExtra.extend(["readmitted"])

print len(colsFinal), len(colsFinalExtended), len(colsFinalExtendedExtra)

42 49 54


#### Save

In [527]:
print typeEncounter,df_all[colsFinal].shape
df_all[colsFinal].to_pickle(os.path.join('resources','prepared_clean_data_' + typeEncounter + "_" +  typeDiagnosis  + '.pkl'))
print 'prepared_clean_data_' + typeEncounter + "_" +  typeDiagnosis  + '.pkl'

last (67182, 42)
prepared_clean_data_last_none.pkl


In [528]:
print typeEncounter,df_all[colsFinalExtended].shape
df_all[colsFinalExtended].to_pickle(os.path.join('resources','prepared_clean_data_' + typeEncounter + "_" +  typeDiagnosis + "_extended"   + '.pkl'))
print 'prepared_clean_data_' + typeEncounter + "_" +  typeDiagnosis + "_extended"  + '.pkl'

last (67182, 49)
prepared_clean_data_last_none_extended.pkl


In [529]:
print typeEncounter,df_all[colsFinalExtendedExtra].shape
df_all[colsFinalExtended].to_pickle(os.path.join('resources','prepared_clean_data_' + typeEncounter + "_" +  typeDiagnosis + "_extended_extra"   + '.pkl'))
print 'prepared_clean_data_' + typeEncounter + "_" +  typeDiagnosis + "_extended_extra"  + '.pkl'

last (67182, 54)
prepared_clean_data_last_none_extended_extra.pkl


In [530]:
print colsFinal
print
print colsFinalExtended
print
print colsFinalExtendedExtra

['gender', 'age', 'race_AfricanAmerican', 'race_Caucasian', 'race_Other', 'HbA1c', 'Change', 'time_in_hospital', 'diabetesMed', 'diss_home', 'medSpec_cardio', 'medSpec_Family/GeneralPractice', 'medSpec_InternalMedicine', 'medSpec_surgery', 'adm_src_1', 'adm_src_2', 'adm_src_3', 'adm_src_4', 'adm_src_5', 'adm_src_6', 'adm_src_7', 'adm_src_8', 'adm_src_10', 'adm_src_11', 'adm_src_13', 'adm_src_14', 'adm_src_22', 'adm_src_25', u'adm_1', u'adm_2', u'adm_3', u'adm_4', u'adm_7', 'number_treatment', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'readmitted']

['gender', 'age', 'race_AfricanAmerican', 'race_Caucasian', 'race_Other', 'HbA1c', 'Change', 'time_in_hospital', 'diabetesMed', 'diss_home', 'medSpec_cardio', 'medSpec_Family/GeneralPractice', 'medSpec_InternalMedicine', 'medSpec_surgery', 'adm_src_1', 'adm_src_2', 'adm_src_3', 'adm_src_4', 'adm_src_5', 'adm_src_6', 'adm_src_7', 'adm_src_8', 'ad

#### Creating other cols

In [531]:
typeDiagnosis = "diag_1" #["diag_1", "diag_3"]

In [533]:
cols = colsFinal[:-1]
colsExtended = colsFinalExtendedExtra[:-1]
if typeDiagnosis == "diag_1":
    auxCols = [
            "Diabetis_1","Circulatory_1",'Digestive_1','Genitourinary_1','Poisoning_1','Muscoskeletal_1',
            'Neoplasms_1','Respiratory_1'      
           ]

if typeDiagnosis == "diag_3":    
    auxCols = [
            "Diabetis_3","Circulatory_3",'Digestive_3','Genitourinary_3','Poisoning_3','Muscoskeletal_3',
            'Neoplasms_3','Respiratory_3'       
           ]
    
cols.extend(auxCols)    
cols.extend(["readmitted"])

colsExtended.extend(auxCols)
colsExtended.extend(["readmitted"])
    
print len(cols), len(colsExtended)
print cols
print colsExtended

50 62
['gender', 'age', 'race_AfricanAmerican', 'race_Caucasian', 'race_Other', 'HbA1c', 'Change', 'time_in_hospital', 'diabetesMed', 'diss_home', 'medSpec_cardio', 'medSpec_Family/GeneralPractice', 'medSpec_InternalMedicine', 'medSpec_surgery', 'adm_src_1', 'adm_src_2', 'adm_src_3', 'adm_src_4', 'adm_src_5', 'adm_src_6', 'adm_src_7', 'adm_src_8', 'adm_src_10', 'adm_src_11', 'adm_src_13', 'adm_src_14', 'adm_src_22', 'adm_src_25', u'adm_1', u'adm_2', u'adm_3', u'adm_4', u'adm_7', 'number_treatment', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'Diabetis_1', 'Circulatory_1', 'Digestive_1', 'Genitourinary_1', 'Poisoning_1', 'Muscoskeletal_1', 'Neoplasms_1', 'Respiratory_1', 'readmitted']
['gender', 'age', 'race_AfricanAmerican', 'race_Caucasian', 'race_Other', 'HbA1c', 'Change', 'time_in_hospital', 'diabetesMed', 'diss_home', 'medSpec_cardio', 'medSpec_Family/GeneralPractice', 'medSpec_InternalM

#### Save

In [534]:
print typeEncounter, df_all[cols].shape
df_all[cols].to_pickle(os.path.join('resources','prepared_clean_data_' + typeEncounter + "_" +  typeDiagnosis + '.pkl'))

last (67182, 50)


In [535]:
print typeEncounter,df_all[colsExtended].shape
df_all[colsExtended].to_pickle(os.path.join('resources','prepared_clean_data_' + typeEncounter + "_" +  typeDiagnosis + '_extended.pkl'))

last (67182, 62)
